In [ ]:
!pip install bert-for-tf2

     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=1faf10f55d2072486d8098c45f07d09ef26fda2de2ddad439836525d98301e77
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=d7a04552b6950aaf5ca18daa09ba5d6084f13650b24292e5aa2a4dbbf6dbfe30
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=44b114e33e5ea8fb6f2d7b119277fdbdabc476ad22d653313c1927313f5b6bce
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 5.9MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import pickle
import re
import pandas as pd


Data Procees and Pretrain Bert


In [ ]:
TAG_RE = re.compile(r'<[^>]+>')
class dataprocessor:

    def __init__(self,filename='./Reviews_new.csv',neutral=0):
        self.df_original = pd.read_csv(filename,header=1,encoding='utf-8',engine="python",error_bad_lines=False)
        self.df_original=self.df_original.dropna()
        self.df_base = self.data_process_baseline(self.df_original,neutral)


    def data_process_baseline(self,df_original, neutral):

        senti = []
        summary = []
        review = []
        for i, j in df_original.iterrows():
            helpfulness = 0.0
            if int(j[5]) > 0:
                helpfulness = float(int(j[4])/int(j[5]))
            if helpfulness > 0.0:
                if int(j[6]) > 3:
                    senti.append(1)
                    #summary.append(self.tokenize_clean(j[8]))
                    review.append(self.tokenize_clean(j[9]))
                if int(j[6]) < 3:
                    senti.append(0)
                    #summary.append(self.tokenize_clean(j[8]))
                    review.append(self.tokenize_clean(j[9]))

                #consider neutral cases
                if neutral:
                    if int(j[6]) == 3:
                        senti.append(0)
                        #summary.append(self.tokenize_clean(j[8]))
                        review.append(self.tokenize_clean(j[9]))
        data_baseline = {"Sentiment": senti, "Review": review}
        df_baseline = pd.DataFrame(data_baseline, columns=["Sentiment","Review"])
        return df_baseline
    def tokenize_clean(self,sent):

      sentence = remove_tags(sent)

      # Remove punctuations and numbers
      sentence = re.sub('[^a-zA-Z]', ' ', sentence)

      # Single character removal
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

      # Removing multiple spaces
      sentence = re.sub(r'\s+', ' ', sentence)

      return sentence



In [ ]:
 def remove_tags(text):
      return TAG_RE.sub('', text)

In [ ]:

source_data=dataprocessor("Reviews_new.csv").df_base


In [ ]:

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [ ]:
reviews = source_data['Review'].tolist()
y = source_data['Sentiment']
tokenized_reviews = [tokenize_reviews(review) for review in reviews]
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

In [ ]:
import random
random.shuffle(reviews_with_len)

In [ ]:
reviews_with_len.sort(key=lambda x: x[2])

In [ ]:
sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

Batch & Create Data Set

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [ ]:
import math

TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 20
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)


Create CNN Model

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

Apply Bert with Model 

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/5
627/627 [==============================] - 101s 161ms/step - loss: 0.2980 - accuracy: 0.8754
Epoch 2/5
627/627 [==============================] - 101s 161ms/step - loss: 0.1122 - accuracy: 0.9584
Epoch 3/5
627/627 [==============================] - 102s 162ms/step - loss: 0.0591 - accuracy: 0.9802
Epoch 4/5
627/627 [==============================] - 104s 165ms/step - loss: 0.0239 - accuracy: 0.9924
Epoch 5/5
627/627 [==============================] - 103s 164ms/step - loss: 0.0091 - accuracy: 0.9974


Evaluate accuracy

In [ ]:
results = text_model.evaluate(test_data)
print(results)



32/32 [==============================] - 0s 11ms/step - loss: 0.2542 - accuracy: 0.9287
[0.25419896841049194, 0.9287109375]
